In [6]:
pip install lxml
!conda install -c conda-forge geopy --yes 
print('Done')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0



geopy-1.21.0         | 58 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [7]:
import numpy as np
import pandas as pd
import lxml.html as lh

from geopy.geocoders import Nominatim
import requests

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Step 1 - Import Table from Wikipedia as 'List', and convert to 'DataFrame'

In [10]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_list = pd.read_html(url)
df_list = df_list[0]
df1 = pd.DataFrame(df_list)
df1 = df1.sort_values(by=['Postal Code'])
df1.head(5)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
9,M1B,Scarborough,"Malvern, Rouge"
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
27,M1E,Scarborough,"Guildwood, Morningside, West Hill"
36,M1G,Scarborough,Woburn


### Step 2 - Delete Rows with 'Not assigned' for 'Borough', and Group by 'Postal Code' 

In [11]:
df2 = df1[df1.Borough != 'Not assigned']
df2 = df2.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(lambda x: ','.join(x.astype(str))).reset_index()
df2.head(5)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Step 3 - Confirm there are no more entries with 'Not assigned'

In [23]:
df3 = df2[df2.Borough == 'Not assigned']
isempty = df3.empty
if isempty == True:
    print('No Unassigned Neighborhoods')
else:
    print ('Some still Unassigned')

No Unassigned Neighborhoods


### Final DataFrame (Show 20 Rows, Sorted by 'Postal Code')

In [26]:
df2.head(20)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"
